<a href="https://colab.research.google.com/github/erdemustun/dosya/blob/main/WR2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed tradingview-screener
import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

# TvDatafeed nesnesi oluşturma
tv = TvDatafeed()

# Türkiye piyasasındaki tüm sembolleri almak için
try:
    Hisseler = get_all_symbols(market='turkey')
    Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
    Hisseler = sorted(Hisseler)
except Exception as e:
    print(f"Hata alındı: {e}")

def _pr(src, length):
    highest_high = src.rolling(window=length).max()
    lowest_low = src.rolling(window=length).min()
    wpr = 100 * (src - highest_high) / (highest_high - lowest_low)
    return wpr

def variant(type, src, length):
    if type == "SMA":
        return src.rolling(window=length).mean()
    elif type == "EMA":
        return src.ewm(span=length, adjust=False).mean()
    elif type == "WMA":
        weights = np.arange(1, length + 1)
        return src.rolling(window=length).apply(lambda prices: np.dot(prices, weights) / weights.sum(), raw=True)
    elif type == "RMA":
        return src.ewm(alpha=1/length, adjust=False).mean()
    return src

def bollinger_bands(type, src, length, mult):
    basis = variant(type, src, length)
    dev = mult * src.rolling(window=length).std()
    upper = basis + dev
    lower = basis - dev
    return basis, upper, lower

def calculate_momentum(data, window=14):
    return data.diff(window)

def ichimoku(data):
    # Ichimoku hesaplamaları
    data['tenkan_sen'] = (data['high'].rolling(window=9).max() + data['low'].rolling(window=9).min()) / 2
    data['kijun_sen'] = (data['high'].rolling(window=26).max() + data['low'].rolling(window=26).min()) / 2
    data['senkou_span_a'] = ((data['tenkan_sen'] + data['kijun_sen']) / 2).shift(26)
    data['senkou_span_b'] = ((data['high'].rolling(window=52).max() + data['low'].rolling(window=52).min()) / 2).shift(26)
    data['chikou_span'] = data['close'].shift(-26)

def parabolic_sar(data, acceleration=0.02, max_acceleration=0.2):
    # Parabolic SAR hesaplamaları
    sar = np.zeros(len(data))
    sar[0] = data['close'][0]
    is_uptrend = True
    ep = data['low'][0]
    af = acceleration

    for i in range(1, len(data)):
        sar[i] = sar[i - 1] + af * (ep - sar[i - 1])
        if is_uptrend:
            sar[i] = max(sar[i], data['low'][i - 1])
            if data['high'][i] > ep:
                ep = data['high'][i]
                af = min(af + acceleration, max_acceleration)
            if data['low'][i] < sar[i]:
                is_uptrend = False
                sar[i] = ep
                ep = data['low'][i]
                af = acceleration
        else:
            sar[i] = min(sar[i], data['high'][i - 1])
            if data['low'][i] < ep:
                ep = data['low'][i]
                af = min(af + acceleration, max_acceleration)
            if data['high'][i] > sar[i]:
                is_uptrend = True
                sar[i] = ep
                ep = data['high'][i]
                af = acceleration
    data['psar'] = sar

def calculate_pivot_points(data):
    data['pivot'] = (data['high'] + data['low'] + data['close']) / 3
    data['support1'] = (2 * data['pivot']) - data['high']
    data['resistance1'] = (2 * data['pivot']) - data['low']
    data['support2'] = data['pivot'] - (data['high'] - data['low'])
    data['resistance2'] = data['pivot'] + (data['high'] - data['low'])

def scan_stocks(symbols, interval='1d', n_bars=500, wpr_period=25, wpr_smoothing_period=3, wpr_smoothing_type="SMA", bb_period=89, bb_mult=1.0, bb_type="SMA", wma_length=50, momentum_window=14):
    results = []

    for symbol in symbols:
        try:
            # TradingView'dan veri çekme
            data = tv.get_hist(symbol=symbol, exchange='BIST', interval=Interval.in_daily, n_bars=n_bars)
            if data is None or data.empty:
                print(f"{symbol} için veri bulunamadı veya boş.")
                continue

            # Williams %R hesaplama
            data['wpr'] = (_pr(data['close'], wpr_period) + 50.0) * 2.0
            data['wpr_smoothed'] = variant(wpr_smoothing_type, data['wpr'], wpr_smoothing_period)

            # Bollinger Bands hesaplama
            middle, upper, lower = bollinger_bands(bb_type, data['wpr_smoothed'], bb_period, bb_mult)
            data['bb_middle'] = middle
            data['bb_upper'] = upper
            data['bb_lower'] = lower

            # WMA hesaplama
            data['wma'] = variant("WMA", data['close'], wma_length)

            # Momentum hesaplama
            data['momentum'] = calculate_momentum(data['close'], window=momentum_window)

            # Ichimoku ve Parabolic SAR hesaplama
            ichimoku(data)
            parabolic_sar(data)

            # Pivot noktaları hesaplama
            calculate_pivot_points(data)

            # Koşulları kontrol etme
            data['go_long'] = (
                (data['wpr_smoothed'] > data['bb_upper']) &
                (data['close'] > data['wma']) &
                (data['momentum'] > 0) &
                (data['close'] > data['psar']) &
                (data['tenkan_sen'] > data['kijun_sen']) &
                (data['close'] > data['senkou_span_a']) &
                (data['close'] > data['senkou_span_b']) &
                (data['close'] > data['pivot']) # Fiyat pivotu yukarı kestiği durumda
            )
            is_long = data.iloc[-1]['go_long']

            # Anlık olarak hisse ve sinyal durumunu gösterme
            print(f"{symbol}: Long Sinyal: {is_long}")

            if is_long:
                results.append({
                    'Hisse Adı': symbol,
                    'Son Fiyat': data.iloc[-1]['close'],
                    'Sinyal': 'Long'
                })
        except Exception as e:
            print(f"Hata alındı {symbol}: {e}")

    return results

# Tarama işlemini başlatma
results = scan_stocks(Hisseler, interval='1d', n_bars=500)

# Sonuçları gösterme
if results:
    df_results = pd.DataFrame(results)
    print("\nKoşulları sağlayan hisseler:")
    print(df_results)
else:
    print("\nHiçbir hisse tarama koşulunu sağlamıyor.")
